In [3]:
using Pkg
import LsqFit
using LinearAlgebra, BenchmarkTools, Profile, CairoMakie

Pkg.activate("/Users/jiyong/development/Projects/nFit.jl")
using ForwardDiff
using nFit

  Activating project at `~/development/Projects/nFit.jl`


In [4]:
q(x, p) = p[1]*sin(x) + p[2]*x + p[3]*exp(-x/p[4])

#q2(x, p) = @. p[1] * exp(-(x-p[2])^2/p[3]) + p[4]*exp(-x/p[5]) + p[6]
t=0.0:0.02:10.0
data = [q(x, [5.0, 0.2, 5, 2]) for x in t]

data += (rand(length(data)).- 0.5);

p0 = [6.0, 6, 50.0, 5, 1, 1.0]
lm = Fitter(q, [1.0, 1.0, 1, 1], collect(t), data, MaxIter=100)
update_jacobian!(lm)
lm.J


501×4 Matrix{Float64}:
  0.0         0.0   1.0         0.0
  0.0199987   0.02  0.980199    0.019604
  0.0399893   0.04  0.960789    0.0384316
  0.059964    0.06  0.941765    0.0565059
  0.0799147   0.08  0.923116    0.0738494
  0.0998334   0.1   0.904837    0.0904838
  0.119712    0.12  0.88692     0.106431
  0.139543    0.14  0.869358    0.12171
  0.159318    0.16  0.852144    0.136343
  0.17903     0.18  0.83527     0.150349
  ⋮                             
 -0.403393    9.84  5.32773e-5  0.000524253
 -0.421612    9.86  5.22223e-5  0.000514916
 -0.439662    9.88  5.11883e-5  0.000505744
 -0.457536    9.9   5.01747e-5  0.000496733
 -0.475227    9.92  4.91812e-5  0.000487881
 -0.492728    9.94  4.82073e-5  0.000479184
 -0.510032    9.96  4.72527e-5  0.000470641
 -0.527132    9.98  4.63171e-5  0.000462248
 -0.544021   10.0   4.53999e-5  0.000454003

In [6]:
@btime fit(lm, 1)

  851.538 μs (32073 allocations: 786.91 KiB)


([5.022361223505638, 0.1996109848936286, 4.9062044537450955, 2.0033037881497378], 100)

In [7]:
lm.J

501×4 Matrix{Float64}:
  0.0         0.0   1.0         0.0
  0.0199987   0.02  0.990066    0.0242073
  0.0399893   0.04  0.980231    0.0479337
  0.059964    0.06  0.970494    0.0711862
  0.0799147   0.08  0.960853    0.0939721
  0.0998334   0.1   0.951308    0.116298
  0.119712    0.12  0.941858    0.138172
  0.139543    0.14  0.932501    0.159599
  0.159318    0.16  0.923238    0.180587
  0.17903     0.18  0.914067    0.201142
  ⋮                             
 -0.403393    9.84  0.0073586   0.0885201
 -0.421612    9.86  0.0072855   0.0878189
 -0.439662    9.88  0.00721312  0.0871229
 -0.457536    9.9   0.00714147  0.0864321
 -0.475227    9.92  0.00707053  0.0857463
 -0.492728    9.94  0.00700029  0.0850657
 -0.510032    9.96  0.00693075  0.0843901
 -0.527132    9.98  0.0068619   0.0837196
 -0.544021   10.0   0.00679374  0.083054

In [8]:

qs(xs, p) = [lm.f(x, p) for x in xs]
ForwardDiff.jacobian(p ->qs(lm.xarr, p), lm.p)

501×4 Matrix{Float64}:
  0.0         0.0   1.0         0.0
  0.0199987   0.02  0.990066    0.0242073
  0.0399893   0.04  0.980231    0.0479336
  0.059964    0.06  0.970494    0.0711862
  0.0799147   0.08  0.960853    0.0939721
  0.0998334   0.1   0.951308    0.116298
  0.119712    0.12  0.941858    0.138172
  0.139543    0.14  0.932501    0.159599
  0.159318    0.16  0.923238    0.180587
  0.17903     0.18  0.914067    0.201142
  ⋮                             
 -0.403393    9.84  0.0073586   0.0885201
 -0.421612    9.86  0.0072855   0.0878189
 -0.439662    9.88  0.00721312  0.0871229
 -0.457536    9.9   0.00714147  0.0864321
 -0.475227    9.92  0.00707053  0.0857463
 -0.492728    9.94  0.00700029  0.0850657
 -0.510032    9.96  0.00693075  0.0843901
 -0.527132    9.98  0.0068619   0.0837196
 -0.544021   10.0   0.00679374  0.083054

In [14]:
# function update_jacobian!(lm::Fitter)
#     qs(xs, p) = [lm.f(x, p) for x in xs]
#     lm.J = ForwardDiff.jacobian(p ->qs(lm.xarr, p), lm.p)

# end
# function update_jacobian2!(lm::Fitter)
#     for j ∈ eachindex(lm.p), i ∈ eachindex(lm.xarr)
#         p1, p2 = lm.p[:], lm.p[:]
#         p1[j] += lm.δ
#         p2[j] -= lm.δ
#         @inbounds lm.J[i, j] = (lm.f(lm.xarr[i], p1) - lm.f(lm.xarr[i], p2))/(2*lm.δ)
        
#     end
# end

@btime begin
p = [6.0, 6, 50.0, 5, 1]
@. qs(xs, p) = [p[1] * exp(-(x-p[2])^2/p[3]) + p[4]*exp(-x/p[5]) for x in xs]
ForwardDiff.jacobian(p ->qs(t, p), p)
end



  20.007 μs (12 allocations: 43.91 KiB)


501×5 Matrix{Float64}:
 0.486752  -0.700923  0.0420554  1.0         0.0
 0.48909   -0.701942  0.0419762  0.980199    0.0980199
 0.491432  -0.702944  0.0418955  0.960789    0.192158
 0.493777  -0.703928  0.0418133  0.941765    0.282529
 0.496125  -0.704894  0.0417297  0.923116    0.369247
 0.498476  -0.705842  0.0416447  0.904837    0.452419
 0.50083   -0.706772  0.0415582  0.88692     0.532152
 0.503188  -0.707683  0.0414702  0.869358    0.608551
 0.505548  -0.708576  0.0413809  0.852144    0.681715
 0.507912  -0.709451  0.04129    0.83527     0.751743
 ⋮                                           
 0.744597   0.686221  0.0263509  5.32773e-5  0.00262124
 0.742307   0.687673  0.0265442  5.22223e-5  0.00257456
 0.740013   0.6891    0.0267371  5.11883e-5  0.0025287
 0.737713   0.6905    0.0269295  5.01747e-5  0.00248365
 0.735409   0.691873  0.0271214  4.91812e-5  0.00243939
 0.733101   0.69322   0.0273129  4.82073e-5  0.0023959
 0.730788   0.694541  0.0275038  4.72527e-5  0.00235319
 0.72

In [10]:
@btime update_jacobian!(lm);
lm.J

  800.373 μs (32068 allocations: 782.86 KiB)


501×4 Matrix{Float64}:
  0.0         0.0   1.0         0.0
  0.0199987   0.02  0.990066    0.0242073
  0.0399893   0.04  0.980231    0.0479337
  0.059964    0.06  0.970494    0.0711862
  0.0799147   0.08  0.960853    0.0939721
  0.0998334   0.1   0.951308    0.116298
  0.119712    0.12  0.941858    0.138172
  0.139543    0.14  0.932501    0.159599
  0.159318    0.16  0.923238    0.180587
  0.17903     0.18  0.914067    0.201142
  ⋮                             
 -0.403393    9.84  0.0073586   0.0885201
 -0.421612    9.86  0.0072855   0.0878189
 -0.439662    9.88  0.00721312  0.0871229
 -0.457536    9.9   0.00714147  0.0864321
 -0.475227    9.92  0.00707053  0.0857463
 -0.492728    9.94  0.00700029  0.0850657
 -0.510032    9.96  0.00693075  0.0843901
 -0.527132    9.98  0.0068619   0.0837196
 -0.544021   10.0   0.00679374  0.083054

In [22]:
@btime curve_fit(q, collect(t), data, [1.0, 1.0, 1, 1])

  6.997 ms (232211 allocations: 6.25 MiB)


([5.022361223505638, 0.1996109848936286, 4.9062044537450955, 2.0033037881497378], 7)

In [23]:
@btime LsqFit.curve_fit(qs, collect(t), data, [1.0, 2, 20.0, 4, 3])

  4.077 ms (110279 allocations: 2.63 MiB)


LsqFit.LsqFitResult{Vector{Float64}, Vector{Float64}, Matrix{Float64}, Vector{Float64}, Vector{LsqFit.LMState{LsqFit.LevenbergMarquardt}}}([5.0249671565176355, 0.19781826256756138, 4.911247511627092, 2.0442995960172117, 3.0], [-0.5885939784839218, 0.16722335515696685, -0.06925195644053783, 0.31392561706300093, 0.34748668210430367, -0.09339941086219827, 0.2855807985645056, -0.3396327003310473, 0.3684158364517911, -0.23540854178097614  …  -0.401035336454104, 0.09644193937765934, 0.3936698549512412, 0.37145299362416223, -0.012282260799746814, -0.5191238233796471, 0.10658587645499845, 0.18700404888176747, 0.37102859331937843, -0.2555856320442058], [0.0 0.0 … 0.0 0.0; 0.019998666694203374 0.01999999998873715 … 0.023274689602150612 0.0; … ; -0.5271319984490501 9.980000000008456 … 0.08893344368101243 0.0; -0.5440211108884259 10.000000000015527 … 0.08824411137652147 0.0], true, Iter     Function value   Gradient norm 
------   --------------   --------------
, Float64[])

In [18]:
ps = LsqFit.curve_fit(qs, collect(t), data, [1.0, 2, 20.0, 4])

LsqFit.LsqFitResult{Vector{Float64}, Vector{Float64}, Matrix{Float64}, Vector{Float64}, Vector{LsqFit.LMState{LsqFit.LevenbergMarquardt}}}([5.0249671565176355, 0.19781826256756138, 4.911247511627092, 2.0442995960172117], [-0.5885939784839218, 0.16722335515696685, -0.06925195644053783, 0.31392561706300093, 0.34748668210430367, -0.09339941086219827, 0.2855807985645056, -0.3396327003310473, 0.3684158364517911, -0.23540854178097614  …  -0.401035336454104, 0.09644193937765934, 0.3936698549512412, 0.37145299362416223, -0.012282260799746814, -0.5191238233796471, 0.10658587645499845, 0.18700404888176747, 0.37102859331937843, -0.2555856320442058], [0.0 0.0 0.9999999999988378 0.0; 0.019998666694203374 0.01999999998873715 0.9902643989738463 0.023274689602150612; … ; -0.5271319984490501 9.980000000008456 0.0075828389457304485 0.08893344368101243; -0.5440211108884259 10.000000000015527 0.007509015450370459 0.08824411137652147], true, Iter     Function value   Gradient norm 
------   -------------- 

In [19]:
ps.param

4-element Vector{Float64}:
 5.0249671565176355
 0.19781826256756138
 4.911247511627092
 2.0442995960172117